## Algoritmia
### Práctica 9
En esta práctica se implementará algún método de *programación dinámica*.

Se pide la implementación de las clases y/o funciones que aparecen a continuación.

Las instrucción "pass" que aparecen en el cuerpo de las clases o funciones, se debe sustituir por la implementación adecuada.

Para cada clase o función que se pide se proporciona una o más funciones con algunos tests.

Al llamar a las funciones de test no debería saltar ninguna aserción.

### Subsecuencia común más larga

In [22]:
def mas_larga(x, y):
    if len(x) > len(y):
        return x
    else:
        return y

In [23]:
from functools import lru_cache
@lru_cache(10000)

def subsecuencia_comun_mas_larga(x, y):
    """
    Dadas dos cadenas x e y devuelve una que es subsecuencia de ambas y que 
    tiene la longitud máxima de todas las subsecuencias comunes.
    """
    if len(x) == 0 or len(y) == 0:
        return []
    if x[-1] == y[-1]:
        return subsecuencia_comun_mas_larga(x[:-1], y[:-1]) + [x[-1]]
    else:
        return mas_larga(subsecuencia_comun_mas_larga(x, y[:-1]), subsecuencia_comun_mas_larga(x[:-1], y))

In [24]:
def test_subsecuencia_comun_mas_larga():
    """Casos de prueba para subsecuencia_comun_mas_larga."""
    
    for s1, s2, l in (
        ("GTTCCTAATA", "CGATAATTGAGA", 6),
        ("ACDAADDADDDDCCBCBCAD", "ADBDBBCDBDAABBDDDCBB", 11),
        ("BBDABCCADCCADADDCACAACBA", "DBCBBDCBADABBBCCCDCACAADDACADD", 17)
    ):    
        for x, y in ((s1, s2), (s2, s1)):
            sub = subsecuencia_comun_mas_larga(x, y)
            assert len(sub) == l
            for s in x, y:
                # comprobamos que sub es subsecuencia de s:
                it = iter(s)
                assert all(c in it for c in sub)
    
if __name__ == "__main__": 
    test_subsecuencia_comun_mas_larga()
    print("OK")    

OK


### Multiplicación encadenada de matrices
Dadas varias matrices, caracterizadas por sus dimensiones, se determinará qué multiplicaciones realizar.

In [25]:
def matrices_encadenadas(r):
    n=len(r)-1
    lista = []
    m=[[0 for i in range(n+1)] for i in range(n+1)]
    imp=[[0 for i in range(n+1)] for i in range(n+1)]
    for d in range(1,n):
        for i in range(1,n-d+1):
            j=d+i
            for k in range(i,j):
                producto=m[i][k]+m[k+1][j]+r[i-1]*r[k]*r[j]
                if k==i or producto < minimo:
                    minimo=producto
                    kmin=k
            #nos quedamos con el minimo, ademas de quedarnos en otra matriz en la que guardamos tres cosas
            #los indices del primer y segundo sumando, y los indices de la multiplicación
            #los dos primeros restados porque empezamos en 1
            m[i][j]=minimo
            lista.append((i-1,kmin-1))
            lista.append((kmin,j-1))
            lista.append((i-1,kmin,j))
            imp[i][j]=lista
            lista = []
    #tuvimos problemas con el tamaño de la matriz y tuvimos que eliminar tanto la primera fila como la segunda
    imp.pop(0)
    m.pop(0)
    for x in range(n):
        imp[x].pop(0)
        m[x].pop(0)
    return m[0][n-1],imp,m

In [26]:
def imprimir(impresion,matrizoriginal,matrizprod):
    #guardamos las tres tuplas guardadas en el anterior de los metodos
    izquierda=impresion[0]
    debajo=impresion[1]
    camino=impresion[2]
    cadena=""
    if matrizoriginal[izquierda[0]][izquierda[1]]==0 and matrizoriginal[debajo[0]][debajo[1]]==0:
        cadena+="M_"+str(camino[0]+1)+"*"+"M_"+str(camino[1]+1)
    elif matrizoriginal[izquierda[0]][izquierda[1]]==0:
        cadena+="M_"+str(camino[0]+1)+"*("+imprimir(matrizprod[debajo[0]][debajo[1]],matrizoriginal,matrizprod)+")"
    elif matrizoriginal[debajo[0]][debajo[1]]==0:
        cadena+="("+imprimir(matrizprod[izquierda[0]][izquierda[1]],matrizoriginal,matrizprod)+")*M_"+str(camino[1]+1)
    else:
        cadena+="("+imprimir(matrizprod[izquierda[0]][izquierda[1]],matrizoriginal,matrizprod)+")*("+imprimir(matrizprod[debajo[0]][debajo[1]],matrizoriginal,matrizprod)+")"
    return cadena

In [27]:
def multiplicacion_encadenada_matrices(dimensiones):
    """
    Dadas las dimensiones de varias matrices a multiplicar, aplica el método
    de programación dinámica para para determinar en qué orden realizar las
    multiplicaciones.
    El número de matrices será la longitud de dimensiones menos uno.
    Las dimensiones de la matriz M_i están en las componentes i-1 e i de
    'dimensiones'.
    Devuelve el número de multiplicaciones de elementos a realizar y una
    cadena con la fórmula, incluyendo paréntesis (solo si son necesarios), en
    la que se realizarían las multiplicaciones.
    Por ejemplo '(M_1*(M_2*M_3))*M_4'.
    """
    res1,temp,temp2 = matrices_encadenadas(dimensiones)
    #ponemos la 0 -1 para coger la de arriba a la derecha del todo ya que hay que empezar a mirar por ahi
    res2=imprimir(temp[0][-1],temp2,temp)
    return res1,res2

In [28]:
def test_orden_multiplicacion_matrices():
    """Casos de prueba para multiplicacion_encadenada_matrices."""
    
    for dimensiones, multiplicaciones, formula in (
        ([13, 5, 89], 13 * 5 * 89, "M_1*M_2"),
        ([13, 5, 89, 3, 34], 2856, "(M_1*(M_2*M_3))*M_4"), 
        ([2, 3, 5, 2, 4, 3], 78, "(M_1*(M_2*M_3))*(M_4*M_5)"),
        ([66, 87, 71, 43, 18, 19, 33, 98, 54], 498402, 
         "(M_1*(M_2*(M_3*M_4)))*(((M_5*M_6)*M_7)*M_8)")
    ):
        assert (multiplicacion_encadenada_matrices(dimensiones) ==
                (multiplicaciones, formula))   
    
if __name__ == "__main__": 
    test_orden_multiplicacion_matrices()
    print("OK")    

OK
